## HinSage  on Amazon Dataset

In [1]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

import stellargraph as sg
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, link_regression
from tensorflow.keras import Model, optimizers, losses, metrics

import multiprocessing
from stellargraph import datasets
from stellargraph import StellarGraph
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import sys

In [2]:
sys.path.append("/Users/msinghal/SourceCodes/Personal/graphranko/GraphicoRango/steller_graph_implementation")
from datasets import Amz_Dataset
from model import GraphicoRango

In [3]:
??Amz_Dataset

# Load Dataset

In [4]:
dataset = Amz_Dataset()
users, items, edge_ratings = dataset.load(base_path="/Users/msinghal/SourceCodes/Personal/graphranko/Data/amazon-electronics-v3")



Index(['item', 'user', 'rating'], dtype='object')
Index(['user', 'user_id'], dtype='object')


In [5]:
print("number of users {},  number of items {}, number of user item rating {}".format(users.shape, items.shape, edge_ratings.shape))

number of users (5000, 1),  number of items (67226, 1745), number of user item rating (168252, 1750)


In [6]:
edge_ratings.head()

,item,user,rating,user_id,item_id,Carry on its own or as a complete protection inside another computer bag,Step-Down Rings,Car Safety & Security,Camcorder Batteries,Computers & Accessories,...,Attachable Shoulder Strap Included (adjustable messenger style strap),SLR Camera Lenses,"Easy Bluetooth Pairing, No Dongle Needed",Headsets,Computer & Monitor Mounts,"With a clean modern look, slender profile and the iconic yet forward new Alien ware industrial design, the Vindicator Slim Carrying Case leaves a lasting impression wherever it goes.","MUTE WHEELS: the Rolling Laptop Bag using high quality universal wheels, this rolling laptop case won't generate too much noise while moving on the floor or even uneven surfaces",36.1 centimeters high,Filters,Dual elastic Touch fastener straps securely hold your tablet in place and prevent accidental drops
0,0151004714,A1ER5AYS3FQ9O3,5.0,u_458,i_44961,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B00006HCJI,A1ER5AYS3FQ9O3,5.0,u_458,i_61201,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,B00006HCJI,A14ME4FQBNFYWH,4.0,u_68,i_61201,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B00029ZE2S,A1ER5AYS3FQ9O3,5.0,u_458,i_56222,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B00029ZE2S,A14QOPWXLIA7A8,5.0,u_829,i_56222,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Split the edge dataset for train and test dataset using sklearn library

In [7]:
from sklearn import model_selection
edges_train, edges_test = model_selection.train_test_split(
    edge_ratings, train_size=0.7, test_size=0.3
)

In [8]:
edgelist_train = list(edges_train[["user_id", "item_id"]].itertuples(index=False))
edgelist_test = list(edges_test[["user_id", "item_id"]].itertuples(index=False))

labels_train = edges_train["rating"]
labels_test = edges_test["rating"]

In [9]:
print("training dataset {}".format(edges_train.shape))
print("test dataset {}".format(edges_test.shape))

training dataset (117776, 1750)
test dataset (50476, 1750)


# Running GraphSage Model without MetaData

## Graph Generation for Graph Neural Network Model

In [10]:
??dataset

In [11]:
graph_with_one_hoting_encoding = dataset.create_graph_for_training(users, items, edge_ratings, metadata=False)

In [12]:
print("--------Numer of Node and edges in Graph -----------")
print("Number of nodes in networkx graph {}, number of eges {}".format(len(graph_with_one_hoting_encoding.nodes()), len(graph_with_one_hoting_encoding.edges())))

--------Numer of Node and edges in Graph -----------
Number of nodes in networkx graph 72226, number of eges 163090


In [13]:
print("---- User Node Features --------")
graph_with_one_hoting_encoding.node_features(node_type = 'user').shape

---- User Node Features --------


(5000, 5000)

In [14]:
print("---- Item Node Features --------")
graph_with_one_hoting_encoding.node_features(node_type = 'item').shape

---- Item Node Features --------


(67226, 67226)

## Compile GraphicoRango Model

In [15]:
batch_size = 200
epochs = 20
num_samples = [8, 4]
graphicoranko_without_metadata = GraphicoRango(edgelist_train, edgelist_test, labels_train, labels_test, epochs, batch_size, graph_with_one_hoting_encoding, num_samples)

In [16]:
graphicoranko_without_metadata.compile()

{'user': [EdgeType(n1='user', rel='default', n2='item')], 'item': [EdgeType(n1='item', rel='default', n2='user')]}
link_regression: using 'concat' method to combine node embeddings into edge embeddings
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 8, 67226)]   0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 32, 5000)]   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 32, 67226)]  0                                            
__________________________________________________________________________________________

## Train Model

In [17]:
## Before training Model accuracy

test_metrics = graphicoranko_without_metadata.model.evaluate(
    graphicoranko_without_metadata.test_gen, verbose=1, use_multiprocessing=False, workers=4
)

print("Untrained model's Test Evaluation:")
for name, val in zip(graphicoranko_without_metadata.model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

253/253 [==============================] - 905s 4s/step - loss: 20.5768 - root_mean_square_error: 4.5358 - mean_absolute_error: 4.4374
Untrained model's Test Evaluation:
	loss: 20.5207
	root_mean_square_error: 4.5294
	mean_absolute_error: 4.4297


In [18]:
history = graphicoranko_without_metadata.train_model()

Epoch 1/20
589/589 [==============================] - 4128s 7s/step - loss: 0.9413 - root_mean_square_error: 0.9347 - mean_absolute_error: 0.6949 - val_loss: 0.7733 - val_root_mean_square_error: 0.8760 - val_mean_absolute_error: 0.6073
Epoch 2/20
589/589 [==============================] - 4077s 7s/step - loss: 0.7118 - root_mean_square_error: 0.8403 - mean_absolute_error: 0.5713 - val_loss: 0.7767 - val_root_mean_square_error: 0.8782 - val_mean_absolute_error: 0.6020
Epoch 3/20
589/589 [==============================] - 3985s 7s/step - loss: 0.6297 - root_mean_square_error: 0.7899 - mean_absolute_error: 0.5216 - val_loss: 0.8169 - val_root_mean_square_error: 0.9012 - val_mean_absolute_error: 0.6418
Epoch 4/20
589/589 [==============================] - 3943s 7s/step - loss: 0.5751 - root_mean_square_error: 0.7551 - mean_absolute_error: 0.4932 - val_loss: 0.8439 - val_root_mean_square_error: 0.9161 - val_mean_absolute_error: 0.6504
Epoch 5/20
589/589 [==============================] - 39

KeyboardInterrupt: 

In [ ]:
sg.utils.plot_history(history)

In [ ]:
## After Model Training Accuracy

test_metrics = graphicoranko_without_metadata.model.evaluate(
    graphicoranko_without_metadata.test_gen, verbose=1, use_multiprocessing=False, workers=4
)

print("Trained model's Test Evaluation:")
for name, val in zip(graphicoranko_without_metadata.model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

## Model Validation

In [ ]:
graphicoranko_without_metadata.validate_model_accuracy()

In [ ]:
y_true = labels_test
# Predict the rankings using the model:
y_pred = graphicoranko_without_metadata.model.predict(graphicoranko_without_metadata.test_gen)
# Mean baseline rankings = mean movie ranking:
y_pred_baseline = np.full_like(y_pred, 3)

rmse = np.sqrt(mean_squared_error(y_true, y_pred_baseline))
mae = mean_absolute_error(y_true, y_pred_baseline)
print("Mean Baseline Test set metrics:")
print("\troot_mean_square_error = ", rmse)
print("\tmean_absolute_error = ", mae)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
print("\nModel Test set metrics:")
print("\troot_mean_square_error = ", rmse)
print("\tmean_absolute_error = ", mae)

In [ ]:
h_true = plt.hist(y_true, bins=30, facecolor="green", alpha=0.5)
h_pred = plt.hist(y_pred, bins=30, facecolor="blue", alpha=0.5)
plt.xlabel("ranking")
plt.ylabel("count")
plt.legend(("True", "Predicted"))
plt.show()

In [ ]:
np.mean(y_true[y_true == 5.0])

# Running HinSage Model with "category" as metadata 

## Graph Generation for Graph Neural Network Models


In [ ]:
graph_with_meta_encoding = dataset.create_graph_for_training(users, items, edge_ratings, metadata=True)


In [ ]:
print("--------Numer of Node and edges in Graph -----------")
print("Number of nodes in networkx graph {}, number of eges {}".format(len(graph_with_meta_encoding.nodes()), len(graph_with_meta_encoding.edges())))

In [ ]:
print("---- User Node Features --------")
graph_with_meta_encoding.node_features(node_type = 'user').shape

In [ ]:
print("---- Item Node Features --------")
graph_with_meta_encoding.node_features(node_type = 'item').shape

## Compile GraphicoRango Model with Metadata

In [ ]:
batch_size = 200
epochs = 20
num_samples = [8, 4]
graphicoranko_with_metadata = GraphicoRango(edgelist_train, edgelist_test, labels_train, labels_test, epochs, batch_size, graph_with_one_hoting_encoding, num_samples)

In [ ]:
graphicoranko_with_metadata.compile()

## Train Model

In [ ]:
## Before training Model accuracy

test_metrics = graphicoranko_with_metadata.model.evaluate(
    graphicoranko_with_metadata.test_gen, verbose=1, use_multiprocessing=False, workers=4
)

print("Untrained model's Test Evaluation:")
for name, val in zip(graphicoranko_with_metadata.model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
history = graphicoranko_with_metadata.train_model()

In [ ]:
# Print History
sg.utils.plot_history(history)

In [ ]:
## After Model Training Accuracy

test_metrics = graphicoranko_with_metadata.model.evaluate(
    graphicoranko_with_metadata.test_gen, verbose=1, use_multiprocessing=False, workers=4
)

print("Trained model's Test Evaluation:")
for name, val in zip(graphicoranko_with_metadata.model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

## Model Validation

In [ ]:
graphicoranko_with_metadata.validate_model_accuracy()

In [ ]:
y_true = labels_test
# Predict the rankings using the model:
y_pred = graphicoranko_with_metadata.model.predict(graphicoranko_with_metadata.test_gen)
# Mean baseline rankings = mean movie ranking:
y_pred_baseline = np.full_like(y_pred, 3)

rmse = np.sqrt(mean_squared_error(y_true, y_pred_baseline))
mae = mean_absolute_error(y_true, y_pred_baseline)
print("Mean Baseline Test set metrics:")
print("\troot_mean_square_error = ", rmse)
print("\tmean_absolute_error = ", mae)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
print("\nModel Test set metrics:")
print("\troot_mean_square_error = ", rmse)
print("\tmean_absolute_error = ", mae)

In [ ]:
h_true = plt.hist(y_true, bins=30, facecolor="green", alpha=0.5)
h_pred = plt.hist(y_pred, bins=30, facecolor="blue", alpha=0.5)
plt.xlabel("ranking")
plt.ylabel("count")
plt.legend(("True", "Predicted"))
plt.show()